In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#  use Image Data Generator to reshape images to avoid overfitting
# 1. / 255 -> scaling
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# class mode = 'binary' because of the binary outcome. for more outcomes -> category
training_set = train_datagen.flow_from_directory('../datasets/cats_dogs_images/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [3]:
# Image Data Generator for the test set -> rescale only!
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('../datasets/cats_dogs_images/test_set',
    # use same target size and batch size as on training set
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [4]:
cnn = tf.keras.models.Sequential()

2023-04-05 12:00:45.780527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Add layers

In [5]:
# add convolutional layers
# filters -> number of feature detectors
# kernel size -> dimensions 3x3
# input shape 64, 64 pixels + 3 of rgb channels
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# create pooling
# pool_size -> 2x2 pixels (picks the max value from the feature frame)
# strides -> shift by 2 pixels 
# paddind: 'valid' (no padding/default), 'same' use padding and don't calculate the vslues outside the square)
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# add 2nd convolutional layer with the pool
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# flattening
cnn.add(tf.keras.layers.Flatten())

Connect layers and add an output layer

In [6]:
# bigger number of units since we deal with images 64x64
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# units = 1 and activation sigmoid (for binary output), softmax for many categories
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile and train

In [7]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
cnn.fit(x = training_set, validation_data=test_set, epochs=20)

Epoch 1/20
250/250 [==============================] - 40s 157ms/step - loss: 0.6896 - accuracy: 0.5376 - val_loss: 0.7049 - val_accuracy: 0.5415
Epoch 2/20
250/250 [==============================] - 34s 137ms/step - loss: 0.6557 - accuracy: 0.6161 - val_loss: 0.6406 - val_accuracy: 0.6350
Epoch 3/20
250/250 [==============================] - 32s 127ms/step - loss: 0.6268 - accuracy: 0.6500 - val_loss: 0.6262 - val_accuracy: 0.6550
Epoch 4/20
250/250 [==============================] - 32s 127ms/step - loss: 0.6051 - accuracy: 0.6721 - val_loss: 0.6282 - val_accuracy: 0.6605
Epoch 5/20
250/250 [==============================] - 32s 127ms/step - loss: 0.5733 - accuracy: 0.7044 - val_loss: 0.6163 - val_accuracy: 0.6670
Epoch 6/20
250/250 [==============================] - 34s 135ms/step - loss: 0.5405 - accuracy: 0.7236 - val_loss: 0.5419 - val_accuracy: 0.7360
Epoch 7/20
250/250 [==============================] - 36s 144ms/step - loss: 0.5056 - accuracy: 0.7469 - val_loss: 0.5552 - val_ac

In [16]:
test1 = tf.keras.utils.load_img('../datasets/cats_dogs_images/imgs/1.jpg', target_size=(64, 64))
test1 = tf.keras.utils.img_to_array(test1)
test1 = np.expand_dims(test1, axis=0)
result1 = cnn.predict(test1)

1/1 [==============================] - 0s 98ms/step


In [18]:
result1

array([[1.]], dtype=float32)

In [26]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [22]:
result1[0][0]

1.0

In [23]:
test2 = tf.keras.utils.load_img('../datasets/cats_dogs_images/imgs/2.jpg', target_size=(64, 64))
test2 = tf.keras.utils.img_to_array(test2)
test2 = np.expand_dims(test2, axis=0)
result2 = cnn.predict(test2)

1/1 [==============================] - 0s 20ms/step


In [24]:
if result2 == 0:
    print('Cat')
else:
    print('Dog')

Cat


In [46]:
def get_result(path:str):
    '''
    the function returns prediction cat or dog
    '''
    output = training_set.class_indices
    
    img = tf.keras.utils.load_img(path, target_size=(64, 64))
    img = tf.keras.utils.img_to_array(img)
    # add a dimention for a batch. put it on the 1st place axis=0
    img = np.expand_dims(img, axis=0)
    # alternatively (from Keras API docs)
    # img = np.array([img])  # Convert single image to a batch.
    res = cnn.predict(img)[0][0].astype(np.uint8)
    final = ''
    for key in output:
        if output[key] == res:
            final = key
    
    return final.title()

In [47]:
get_result('../datasets/cats_dogs_images/imgs/3.webp')

1/1 [==============================] - 0s 20ms/step


'Dogs'